In [1]:
import pandas as pd
import sqlalchemy as sa

from decouple import config

from pandas import (
    Int64Dtype,
    StringDtype,
)

from sqlalchemy.dialects.postgresql import (
    INTEGER,
    VARCHAR,
)

In [2]:
brasil_data = pd.read_csv("http://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv")

In [3]:
brasil_data = brasil_data.query("SITUACAO_IES == 'Ativa'")

In [4]:
brasil_data.insert(
    loc = 0,
    column = "university_country",
    value = "BR",
)

In [5]:
brasil_data = brasil_data.rename(
    columns = {
        "CODIGO_DA_IES": "university_id",
        "NOME_DA_IES": "university_name",
        "UF": "university_state",
        "MUNICIPIO": "university_city"
    }
)

In [6]:
brasil_data = brasil_data.reindex(
    columns = [
        "university_id",
        "university_name",
        "university_campus",
        "university_country",
        "university_state",
        "university_city",
    ]
)

In [7]:
brasil_data = brasil_data.astype(
    dtype = {
        "university_id": Int64Dtype(),
        "university_name": StringDtype(),
        "university_campus": StringDtype(),
        "university_country": StringDtype(),
        "university_state": StringDtype(),
        "university_city": StringDtype(),
    }
)

In [8]:
brasil_data["university_name"] = brasil_data["university_name"].str.title()

In [9]:
POSTGRES_URL = config("POSTGRES_URL", default = "sqlite:///university.db")

In [10]:
engine_database = sa.create_engine(POSTGRES_URL)

In [11]:
# ! - One time etl, after that you'll need to erase some data to run this again
with engine_database.connect() as database_connection:
    brasil_data.to_sql(
        name = "rates_universities",
        con = database_connection,
        if_exists = "append",
        dtype = {
            "university_id": INTEGER,
            "university_name": VARCHAR(255),
            "university_campus": VARCHAR(255),
            "university_country": VARCHAR(255),
            "university_state": VARCHAR(255),
            "university_city": VARCHAR(255),
        }
    )